In [ ]:
!pip install langchain-mcp-adapters langgraph langchain-openai

In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

In [ ]:
from pathlib import Path
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    openai_api_base="https://llm.nrp-nautilus.io/",
    openai_api_key=Path("~/.keys/nrp-llm").expanduser().read_text().strip(),
    model = "llama3",
    #model = "groq-tools",
    temperature=0.1
)


In [ ]:
server_params = StdioServerParameters(
    command="python",
    args=[str(Path.cwd() / "pg_server_SHAREOK.py")],
)

In [ ]:
async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)

        # Create and run the agent
        agent = create_react_agent(model, tools)
        agent_response = await agent.ainvoke({"messages": "What documents are about frogs? Give me the author name."})

In [ ]:
agent_response['messages']

In [ ]:
from json import loads
from pprint import pprint
pprint(agent_response['messages'][-1].content)